In [63]:
import pandas as pd
import numpy as np
import re
from google.cloud import bigquery
from google.auth import default
from datetime import datetime, timedelta

In [ ]:
credentials, project = default()
client = bigquery.Client(credentials=credentials, project=project)

job_config = bigquery.QueryJobConfig(
    dry_run=True,
    use_query_cache=False
    )

In [ ]:
crafts = """select * from bigtimestudios.BT_ML.ai_bot_input_crafts"""
dau = """select * from bigtimestudios.BT_ML.ai_bot_input_dau"""
dungeons = """select * from bigtimestudios.BT_ML.ai_bot_input_dungeon"""
revenue = """select * from bigtimestudios.BT_ML.ai_bot_input_revenue"""
token = """select * from bigtimestudios.BT_ML.ai_bot_input_token"""


queries = {
    'crafts': crafts,
    'dau': dau,
    'dungeons': dungeons,
    'revenue': revenue,
    'token': token
}

for name, query in queries.items():
  query_job = client.query(query, job_config=job_config)
  gb_processed = query_job.total_bytes_processed / 1024**3
  print(f'{name} query will process {gb_processed:.2f} GB.')

crafts query will process 0.00 GB.
dau query will process 0.00 GB.
dungeons query will process 0.00 GB.
revenue query will process 0.00 GB.
token query will process 0.00 GB.


In [53]:
## Bigquery helper function
def run_bq(query: str, params: list[bigquery.ScalarQueryParameter] | None = None, max_gb: float = 2.0):
  job_config = bigquery.QueryJobConfig(
      query_parameters=params or [],
      use_query_cache=True,
    )
  dry = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False, query_parameters=params or [])
  dry_job = client.query(query, job_config=dry)
  gb = dry_job.total_bytes_processed / 1024**3
  if gb > max_gb:
    raise ValueError(f'Query blocked: {gb:.2f} GB would be processed (limit {max_gb:.2f}) GB.')
  job = client.query(query, job_config=job_config)
  return [dict(row) for row in job.result()], gb

In [57]:
# get the data
crafts_df = client.query(crafts).to_dataframe()
dau_df = client.query(dau).to_dataframe()
dungeons_df = client.query(dungeons).to_dataframe()
revenue_df = client.query(revenue).to_dataframe()
token_df = client.query(token).to_dataframe()

In [64]:
def _to_date(s):
    return pd.to_datetime(s, errors="coerce").dt.date

def normalize_tables(crafts_df, dau_df, dungeons_df, revenue_df, token_df):
    # crafts_df: Day
    if "Day" in crafts_df.columns:
        crafts_df = crafts_df.copy()
        crafts_df["Day"] = _to_date(crafts_df["Day"])

    # dau_df: checkpoint (could be timestamp or date-like)
    dau_df = dau_df.copy()
    if "checkpoint" in dau_df.columns:
        dau_df["checkpoint_ts"] = pd.to_datetime(dau_df["checkpoint"], errors="coerce")
        # Derive a date column for consistent filtering
        dau_df["Day"] = dau_df["checkpoint_ts"].dt.date

    # dungeons_df: Day
    if "Day" in dungeons_df.columns:
        dungeons_df = dungeons_df.copy()
        dungeons_df["Day"] = _to_date(dungeons_df["Day"])

    # revenue_df: Date
    revenue_df = revenue_df.copy()
    if "Date" in revenue_df.columns:
        revenue_df["Date"] = _to_date(revenue_df["Date"])
        revenue_df["Day"] = revenue_df["Date"]  # unify naming

    # token_df: Day
    token_df = token_df.copy()
    if "Day" in token_df.columns:
        token_df["Day"] = _to_date(token_df["Day"])

    return crafts_df, dau_df, dungeons_df, revenue_df, token_df

crafts_df, dau_df, dungeons_df, revenue_df, token_df = normalize_tables(
    crafts_df, dau_df, dungeons_df, revenue_df, token_df
)

def last_n_days(n: int):
    today = datetime.utcnow().date()
    return today - timedelta(days=n), today

def _filter_last_n(df, n, day_col="Day"):
    start, end = last_n_days(n)
    return df[(df[day_col] >= start) & (df[day_col] <= end)]

def _print_df(df, max_rows=20):
    if df is None or len(df) == 0:
        print("(no rows)")
        return
    display(df.head(max_rows))


In [65]:
def tool_dau_last(n=7):
    df = _filter_last_n(dau_df, n, "Day").sort_values("Day")
    out = df[["Day", "DAU", "hours_played", "avg_time_per_player"]].copy()
    return {"title": f"DAU last {n} days", "data": out}

def tool_dau_on(day_str):
    d = pd.to_datetime(day_str).date()
    df = dau_df[dau_df["Day"] == d]
    out = df[["checkpoint", "DAU", "hours_played", "avg_time_per_player"]].copy()
    return {"title": f"DAU on {d}", "data": out.sort_values("checkpoint")}

def tool_crafts_last(n=7):
    df = _filter_last_n(crafts_df, n, "Day").sort_values("Day")
    agg = df.groupby("Day", as_index=False).agg(
        crafts=("crafts", "sum"),
        users=("users", "sum"),
    )
    agg["crafts_per_user"] = (agg["crafts"] / agg["users"]).round(3)
    return {"title": f"Crafts last {n} days", "data": agg}

def tool_crafts_top_names(n=7, top=10):
    df = _filter_last_n(crafts_df, n, "Day")
    agg = df.groupby("name", as_index=False).agg(
        crafts=("crafts", "sum"),
        users=("users", "sum")
    ).sort_values("crafts", ascending=False).head(top)
    return {"title": f"Top {top} craft names last {n} days", "data": agg}

def tool_dungeons_last(n=7):
    df = _filter_last_n(dungeons_df, n, "Day")
    agg = df.groupby(["portalType", "mission"], as_index=False).agg(
        Players=("Players", "sum"),
        dungeonUptime=("dungeonUptime", "mean"),
        sandConsumed=("sandConsumed", "sum")
    ).sort_values("Players", ascending=False).head(20)
    return {"title": f"Dungeons summary last {n} days (top 20 by Players)", "data": agg}

def tool_dungeons_filter(n=7, portalType=None, mission=None):
    df = _filter_last_n(dungeons_df, n, "Day")
    if portalType:
        df = df[df["portalType"].astype(str).str.contains(portalType, case=False, na=False)]
    if mission:
        df = df[df["mission"].astype(str).str.contains(mission, case=False, na=False)]

    agg = df.groupby(["Day", "portalType", "mission"], as_index=False).agg(
        Players=("Players", "sum"),
        dungeonUptime=("dungeonUptime", "mean"),
        sandConsumed=("sandConsumed", "sum")
    ).sort_values(["Day", "Players"], ascending=[False, False]).head(50)
    return {"title": f"Dungeons filtered last {n} days", "data": agg}

def tool_revenue_last(n=7):
    df = _filter_last_n(revenue_df, n, "Day")
    agg = df.groupby("Day", as_index=False).agg(
        revenue=("revenue", "sum"),
        Buyers=("Buyers", "sum"),
        transactions=("transactions", "sum"),
    ).sort_values("Day")
    agg["rev_per_buyer"] = (agg["revenue"] / agg["Buyers"]).round(2)
    agg["rev_per_txn"] = (agg["revenue"] / agg["transactions"]).round(2)
    return {"title": f"Revenue last {n} days", "data": agg}

def tool_revenue_item(item_name, n=30):
    df = _filter_last_n(revenue_df, n, "Day")
    df = df[df["item_name"].astype(str).str.contains(item_name, case=False, na=False)]
    agg = df.groupby("Day", as_index=False).agg(
        revenue=("revenue", "sum"),
        Buyers=("Buyers", "sum"),
        transactions=("transactions", "sum"),
    ).sort_values("Day")
    return {"title": f'Revenue for item "{item_name}" last {n} days', "data": agg}

def tool_token_last(n=7):
    df = _filter_last_n(token_df, n, "Day")
    agg = df.groupby(["Day", "action"], as_index=False).agg(
        amount=("amount", "sum"),
        users=("users", "sum"),
    ).sort_values(["Day", "amount"], ascending=[False, False]).head(50)
    return {"title": f"Token activity last {n} days (by action)", "data": agg}

def tool_token_action(action, n=7):
    df = _filter_last_n(token_df, n, "Day")
    df = df[df["action"].astype(str).str.contains(action, case=False, na=False)]
    agg = df.groupby(["Day", "action", "sourceString"], as_index=False).agg(
        amount=("amount", "sum"),
        users=("users", "sum"),
    ).sort_values(["Day", "amount"], ascending=[False, False]).head(50)
    return {"title": f'Token action "{action}" last {n} days (by source)', "data": agg}

def tool_token_net(n=7):
    # If 'amount' is signed already, this is true net.
    # If not, this will be misleading — but it’s still a useful first pass.
    df = _filter_last_n(token_df, n, "Day")
    agg = df.groupby("Day", as_index=False).agg(net_amount=("amount", "sum"), users=("users", "sum"))
    return {"title": f"Token net amount last {n} days", "data": agg.sort_values("Day")}


In [ ]:
HELP = """
Commands:
  dau last <n>
  dau on <YYYY-MM-DD>

  crafts last <n>
  crafts top <n> <top>          (e.g. crafts top 7 10)

  dungeons last <n>
  dungeons mission "<text>" last <n>
  dungeons portal "<text>" last <n>

  revenue last <n>
  revenue item "<text>" last <n>

  token last <n>
  token action "<text>" last <n>
  token net last <n>

  help
  exit
""".strip()

def parse_int(s, default):
    try:
        return int(s)
    except:
        return default

def chat():
    print("Notebook bot ready. Type 'help' for commands. Type 'exit' to quit.")
    while True:
        msg = input("> ").strip()
        if not msg:
            continue
        if msg.lower() in ("exit", "quit"):
            break
        if msg.lower() == "help":
            print(HELP)
            continue

        try:
            m = msg.strip()

            # dau
            if re.match(r"^dau\s+last\s+\d+$", m, re.I):
                n = int(m.split()[-1])
                res = tool_dau_last(n)

            elif re.match(r"^dau\s+on\s+\d{4}-\d{2}-\d{2}$", m, re.I):
                day = m.split()[-1]
                res = tool_dau_on(day)

            # crafts
            elif re.match(r"^crafts\s+last\s+\d+$", m, re.I):
                n = int(m.split()[-1])
                res = tool_crafts_last(n)

            elif re.match(r"^crafts\s+top\s+\d+\s+\d+$", m, re.I):
                _, _, n, top = m.split()
                res = tool_crafts_top_names(int(n), int(top))

            # dungeons
            elif re.match(r"^dungeons\s+last\s+\d+$", m, re.I):
                n = int(m.split()[-1])
                res = tool_dungeons_last(n)

            elif re.match(r'^dungeons\s+mission\s+".*"\s+last\s+\d+$', m, re.I):
                mission = re.findall(r'"(.*?)"', m)[0]
                n = int(re.findall(r"last\s+(\d+)$", m, re.I)[0])
                res = tool_dungeons_filter(n=n, mission=mission)

            elif re.match(r'^dungeons\s+portal\s+".*"\s+last\s+\d+$', m, re.I):
                portal = re.findall(r'"(.*?)"', m)[0]
                n = int(re.findall(r"last\s+(\d+)$", m, re.I)[0])
                res = tool_dungeons_filter(n=n, portalType=portal)

            # revenue
            elif re.match(r"^revenue\s+last\s+\d+$", m, re.I):
                n = int(m.split()[-1])
                res = tool_revenue_last(n)

            elif re.match(r'^revenue\s+item\s+".*"\s+last\s+\d+$', m, re.I):
                item = re.findall(r'"(.*?)"', m)[0]
                n = int(re.findall(r"last\s+(\d+)$", m, re.I)[0])
                res = tool_revenue_item(item, n=n)

            # token
            elif re.match(r"^token\s+last\s+\d+$", m, re.I):
                n = int(m.split()[-1])
                res = tool_token_last(n)

            elif re.match(r'^token\s+action\s+".*"\s+last\s+\d+$', m, re.I):
                action = re.findall(r'"(.*?)"', m)[0]
                n = int(re.findall(r"last\s+(\d+)$", m, re.I)[0])
                res = tool_token_action(action, n=n)

            elif re.match(r"^token\s+net\s+last\s+\d+$", m, re.I):
                n = int(m.split()[-1])
                res = tool_token_net(n)

            else:
                print("Unrecognized command. Type 'help'.")
                continue

            print(f"\n{res['title']}")
            _print_df(res["data"])
            print("")

        except Exception as e:
            print(f"Error: {e}")

chat()

Notebook bot ready. Type 'help' for commands. Type 'exit' to quit.
> help
Commands:
  dau last <n>
  dau on <YYYY-MM-DD>

  crafts last <n>
  crafts top <n> <top>          (e.g. crafts top 7 10)

  dungeons last <n>
  dungeons mission "<text>" last <n>
  dungeons portal "<text>" last <n>

  revenue last <n>
  revenue item "<text>" last <n>

  token last <n>
  token action "<text>" last <n>
  token net last <n>

  help
  exit
> dau last
Unrecognized command. Type 'help'.
> dau last <7>
Unrecognized command. Type 'help'.
> dau last <n>
Unrecognized command. Type 'help'.
> dau last <n>
Unrecognized command. Type 'help'.
> dau last 7\
Unrecognized command. Type 'help'.
> dau last 7

DAU last 7 days


,Day,DAU,hours_played,avg_time_per_player
22,2026-01-07,248,1939.828056,7.821887
19,2026-01-09,245,1881.982222,7.681560
25,2026-01-10,258,2092.387222,8.110028
28,2026-01-11,277,2214.054444,7.992976
20,2026-01-12,245,1748.396944,7.136314
15,2026-01-13,243,1807.193611,7.437011
